# **Data Cleaning - Sales Pipeline**

This notebook is designed to perform **data cleaning** on the **sales_pipeline** table that is part of the CRM dataset for the analysis project.

In [136]:
# Import required library
import pandas as pd
import re

In [ ]:
# Load Dataset
team = pd.read_csv('/Users/Gio Noga/Documents/Data Analysis 101/repos/gn-data-crm_pricing_analysis/raw_dataset/sales_pipeline.csv')

# Load issue dataset
issues = pd.read_csv('/Users/Gio Noga/Documents/Data Analysis 101/repos/gn-data-crm_pricing_analysis/dataset_validation/data_quality_check/raw_data_issues.csv')

In [138]:
# Create a copy of the original dataset for cleaning
opportunity_cleaned = team.copy()
opportunity_cleaned.head(3)

,opportunity_id,sales_agent,product,account,deal_stage,engage_date,close_date,close_value
0,1C1I7A6R,Moses Frase,GTX Plus Basic,Cancity,Won,2016-10-20,2017-03-01,1054.0
1,Z063OYW0,Darcel Schlecht,GTXPro,Isdom,Won,2016-10-25,2017-03-11,4514.0
2,EC4QE1BX,Darcel Schlecht,MG Special,Cancity,Won,2016-10-25,2017-03-07,50.0


### **To-Do list based on data validation findings**

In [139]:
# View issues related to sales_teams dataset
pd.options.display.max_colwidth = None
issues.loc[issues['dataset'] == 'sales_pipeline']

,dataset,column,issue,details,action,status
4,sales_pipeline,product,Inconsistent Formatting,Product value 'GTXPro' is missing a space between 'GTX' and 'Pro'.,Standardize product name to 'GTX Pro'.,Resolved
5,sales_pipeline,account,Inconsistent Formatting,The first letter of the value 'dambase' is not capitalized.,Standardize the account names to have the first letter capitalized.,Resolved


### **Fix Formatting Issues Specific Values**

In [140]:
# Capitalize account 'dambase' value
opportunity_cleaned['account'] = (
    opportunity_cleaned['account'].str[:1].str.upper() + 
    opportunity_cleaned['account'].str[1:]
)

In [141]:
# ---------------------------
# Validate account column
# ---------------------------
def validate_account(account, deal_stage):

    # Allow NULL only if deal_stage is Prospecting or Engaging
    if pd.isna(account):
        if deal_stage in ["Prospecting", "Engaging"]:
            return "valid"
        else:
            return "null_not_allowed"

    name = str(account).strip()

    # Rule 1: At least one word
    tokens = name.split()
    if len(tokens) < 1:
        return "no_words"

    # Rule 2: Allowed characters only
    if not re.fullmatch(r"[A-Za-z0-9\s&\-\.\']+", name):
        return "invalid_characters"

    # Rule 3: First letter of first word must be uppercase
    if not tokens[0][0].isupper():
        return "first_word_not_capitalized"

    return "valid"


# ---------------------------
# Run validation (row-wise, NO column persisted)
# ---------------------------
account_validation_results = opportunity_cleaned.apply(
    lambda row: validate_account(row['account'], row['deal_stage']),
    axis=1
)

# ---------------------------
# Summary
# ---------------------------
print("Validation Results:")
print(account_validation_results.value_counts())


# ---------------------------
# Extract issues ONLY (do NOT overwrite dq issues table)
# ---------------------------
opportunity_account_validation_issues = (
    opportunity_cleaned
    .assign(account_validation=account_validation_results)  # temporary only
    .loc[
        account_validation_results != 'valid',
        ['account', 'deal_stage', 'account_validation']
    ]
    .groupby(['account', 'deal_stage', 'account_validation'])
    .size()
    .reset_index(name='affected_rows')
)

if not opportunity_account_validation_issues.empty:
    print("\nIssues found:")
    print(opportunity_account_validation_issues)
else:
    print("\nPassed")

Validation Results:
valid    8800
Name: count, dtype: int64

Passed


In [142]:
# Fix typo in product
opportunity_cleaned.loc[
    opportunity_cleaned['product'] == 'GTXPro',
    'product'
] = 'GTX Pro'

In [143]:
# ---------------------------
# Validate product column
# ---------------------------
def validate_product(product):

    if pd.isna(product):
        return "null_value"

    name = str(product).strip()

    # Rule 1: Excessive internal whitespace
    if "  " in name:
        return "excessive_whitespace"

    # Rule 2: Letters, numbers, and spaces only
    if not re.fullmatch(r"[A-Za-z0-9\s]+", name):
        return "invalid_characters"

    tokens = name.split()

    # Rule 3: Product name must have 2–3 words
    if len(tokens) < 2 or len(tokens) > 3:
        return "invalid_word_count"

    # Rule 4: First word must be all uppercase
    if not tokens[0].isupper():
        return "first_word_not_uppercase"

    return "valid"


# ---------------------------
# Run validation (NO column persisted)
# ---------------------------
product_validation_results = opportunity_cleaned['product'].apply(validate_product)

# ---------------------------
# Summary
# ---------------------------
print("Validation Results:")
print(product_validation_results.value_counts())

# ---------------------------
# Extract issues ONLY (do NOT overwrite dq issues table)
# ---------------------------
product_validation_issues = (
    opportunity_cleaned
    .assign(product_validation=product_validation_results)  # temporary only
    .loc[
        product_validation_results != 'valid',
        ['product', 'product_validation']
    ]
    .groupby(['product', 'product_validation'])
    .size()
    .reset_index(name='affected_rows')
)

if not product_validation_issues.empty:
    print("\nIssues found:")
    print(product_validation_issues)
else:
    print("\nPassed")

Validation Results:
product
valid    8800
Name: count, dtype: int64

Passed


In [ ]:
# Update issues dataset to mark issue as resolved
issues.loc[
    (issues['dataset'] == 'sales_pipeline') &
    (issues['issue'] == 'Inconsistent Formatting'),
    'status'
] = 'Resolved'

issues.to_csv('/Users/Gio Noga/Documents/Data Analysis 101/repos/gn-data-crm_pricing_analysis/dataset_validation/data_quality_check/raw_data_issues.csv', index=False)

pd.options.display.max_colwidth = None
issues[issues['dataset'] == 'sales_pipeline']

,dataset,column,issue,details,action,status
4,sales_pipeline,product,Inconsistent Formatting,Product value 'GTXPro' is missing a space between 'GTX' and 'Pro'.,Standardize product name to 'GTX Pro'.,Resolved
5,sales_pipeline,account,Inconsistent Formatting,The first letter of the value 'dambase' is not capitalized.,Standardize the account names to have the first letter capitalized.,Resolved


## **Remove Uncessesary Columns for Analysis**

In [145]:
opportunity_cleaned.rename(columns={'deal_stage': 'deal_outcome'}, inplace=True)


In [146]:
# Remove rows with deal_outcome other than 'Won' or 'Lost'
opportunity_cleaned = opportunity_cleaned[
    opportunity_cleaned['deal_outcome'].isin(['Won', 'Lost'])
]

In [147]:
# Final check of deal_outcome values
opportunity_cleaned['deal_outcome'].value_counts(dropna=False)

deal_outcome
Won     4238
Lost    2473
Name: count, dtype: int64

In [148]:
# Check for nulls and data types
opportunity_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6711 entries, 0 to 8299
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   opportunity_id  6711 non-null   object 
 1   sales_agent     6711 non-null   object 
 2   product         6711 non-null   object 
 3   account         6711 non-null   object 
 4   deal_outcome    6711 non-null   object 
 5   engage_date     6711 non-null   object 
 6   close_date      6711 non-null   object 
 7   close_value     6711 non-null   float64
dtypes: float64(1), object(7)
memory usage: 471.9+ KB


## Add Feature Columns for Analysis

Now that all issues in the dataset are resolved, several feature column will be added that will be use for analysis. Apart from this unneeded column will be raplace by feature and some rows will be drop since they are now needed for the analysis

In [149]:
opportunity_cleaned.head(3)

,opportunity_id,sales_agent,product,account,deal_outcome,engage_date,close_date,close_value
0,1C1I7A6R,Moses Frase,GTX Plus Basic,Cancity,Won,2016-10-20,2017-03-01,1054.0
1,Z063OYW0,Darcel Schlecht,GTX Pro,Isdom,Won,2016-10-25,2017-03-11,4514.0
2,EC4QE1BX,Darcel Schlecht,MG Special,Cancity,Won,2016-10-25,2017-03-07,50.0


## **Add deal_duration column**

The total number of days between the deal engagement date and close date, representing the exact lifecycle length of a sales opportunity.

In [150]:
# Add deal_duration column
opportunity_cleaned.insert(
    7,
    'deal_duration',
    (
        pd.to_datetime(opportunity_cleaned['close_date'], errors='coerce')
        - pd.to_datetime(opportunity_cleaned['engage_date'], errors='coerce')
    ).dt.days
)

In [151]:
#Get summary statistics
# opportunity_cleaned.describe(include='all')
opportunity_cleaned.info()


<class 'pandas.core.frame.DataFrame'>
Index: 6711 entries, 0 to 8299
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   opportunity_id  6711 non-null   object 
 1   sales_agent     6711 non-null   object 
 2   product         6711 non-null   object 
 3   account         6711 non-null   object 
 4   deal_outcome    6711 non-null   object 
 5   engage_date     6711 non-null   object 
 6   close_date      6711 non-null   object 
 7   deal_duration   6711 non-null   int64  
 8   close_value     6711 non-null   float64
dtypes: float64(1), int64(1), object(7)
memory usage: 524.3+ KB


## **Add a deal_duration_bucket column**

A categorical grouping of deal duration expressed in months, used to segment opportunities into standardized time ranges for reporting and trend analysis.

In [152]:
# Convert deal_duration to months (approximate)
deal_duration_months = opportunity_cleaned['deal_duration'] / 30

# Define bins and labels
bins = [0, 1, 2, 4, float('inf')]  # in months
labels = ['0-1', '2', '3-4', '5+']

# Insert bucket column at position 8
opportunity_cleaned.insert(
    8,
    'deal_duration_bucket_month',
    pd.cut(
        deal_duration_months,
        bins=bins,
        labels=labels,
        right=True,
        include_lowest=True
    )
)


In [153]:
# Validation: check counts per bucket
print(opportunity_cleaned['deal_duration_bucket_month'].value_counts())

deal_duration_bucket_month
0-1    3163
3-4    2786
2       569
5+      193
Name: count, dtype: int64


In [154]:
opportunity_cleaned.head(3)

,opportunity_id,sales_agent,product,account,deal_outcome,engage_date,close_date,deal_duration,deal_duration_bucket_month,close_value
0,1C1I7A6R,Moses Frase,GTX Plus Basic,Cancity,Won,2016-10-20,2017-03-01,132,5+,1054.0
1,Z063OYW0,Darcel Schlecht,GTX Pro,Isdom,Won,2016-10-25,2017-03-11,137,5+,4514.0
2,EC4QE1BX,Darcel Schlecht,MG Special,Cancity,Won,2016-10-25,2017-03-07,133,5+,50.0


## **Add a price_adjustment_pct column**

The percentage difference between the final closed deal value and the product’s standard reference price, indicating whether the won deal was closed at a premium (positive) or a discount (negative).

In [155]:
# Reference dictionary for sales_price
sales_price_ref = {
    'GTX Basic': 550,
    'GTX Pro': 4821,
    'MG Special': 55,
    'MG Advanced': 3393,
    'GTX Plus Pro': 5482,
    'GTX Plus Basic': 1096,
    'GTK 500': 26768
}

# Insert price_adjustment_pct at column 10
opportunity_cleaned.insert(
    10,
    'price_adjustment_pct',
    opportunity_cleaned['product'].map(sales_price_ref)
    .combine(opportunity_cleaned['close_value'], 
             lambda srp, closed: (closed - srp) / srp)
    .where(opportunity_cleaned['deal_outcome'] != 'Lost', 0)
)


In [156]:
opportunity_cleaned.head(3)

,opportunity_id,sales_agent,product,account,deal_outcome,engage_date,close_date,deal_duration,deal_duration_bucket_month,close_value,price_adjustment_pct
0,1C1I7A6R,Moses Frase,GTX Plus Basic,Cancity,Won,2016-10-20,2017-03-01,132,5+,1054.0,-0.038321
1,Z063OYW0,Darcel Schlecht,GTX Pro,Isdom,Won,2016-10-25,2017-03-11,137,5+,4514.0,-0.063680
2,EC4QE1BX,Darcel Schlecht,MG Special,Cancity,Won,2016-10-25,2017-03-07,133,5+,50.0,-0.090909


In [ ]:
# Save the cleaned dataframe to a new CSV file
opportunity_cleaned.to_csv('C:\\Users\\Gio Noga\\Documents\\Data Analysis 101\\repos\\gn-data-crm_pricing_analysis\\clean_dataset/04_sales_pipeline_cleaned.csv', index=False)